Importamos Pandas y Numpy

In [12]:
import pandas as pd
import numpy as np

Then comes connecting to working directories.


In [13]:
import os
# show current working directory
os.getcwd()
# list files in the directory
os.listdir()
# change working directory
os.chdir("/")

#2. Loading data
Now load in your datasets from the repository (desktop, cloud, SQL server — wherever stored). It’s a good idea to make a copy of the original dataset and work with the copy because you’ll be doing a lot of modifications to the original one.

In [24]:
# import from a csv file
#data = pd.read_csv("Iris.csv")
data = pd.read_csv("data.csv")
# copying a dataset
df = data.copy()
# call the head function
df.head()

FileNotFoundError: ignored

#3. Initial data screening


In [ ]:
df.info()

In [ ]:
# number of rows and columns
df.shape
# column names
df.columns


In [ ]:
# number of unique values
df["species"].nunique()
# name of the unique values
df["species"].unique()

In [ ]:
# count of categorical data
df["species"].value_counts()

#4. Missing value treatment
Missing values are no surprise. The yellow highlighted cells in the dataframe above are NaN values. You can search for the number of missing values in a dataset by typing the following:


In [ ]:
# show NaN values per feature
df.isnull().sum()


You could also obtain missing values as a percentage of total observations (it is quite useful for large datasets).


In [ ]:
# NaN values as % of total observations
df.isnull().sum()*100/len(df)

In the iris dataset we have two missing values in the sepal_length column. Now that we found out, what to do with them? You can do one of the following:

a ) drop the rows or columns containing null values;


In [ ]:
## Drop row/column ##
#####################
# drop all rows containing null
df.dropna()
# drop all columns containing null
df.dropna(axis=1)
# drop columns with less than 5 NaN values
df.dropna(axis=1, thresh=5)

b ) or, replace/impute missing cells with some other values;


In [ ]:
## Replace values ##
####################
# replace all na values with -9999
df.fillna(-9999)
[# additional tip: you can also replace any specific cell values
df.at[1, "sepal_length"]= 9999]
# fill na values with NaN
df.fillna(np.NaN)
# fill na values with strings
df.fillna("data missing")
# fill missing values with mean column values
df.fillna(df.mean())
# replace na values of specific columns with mean value
df["sepal_length"].fillna(df["sepal_length"].mean())

c ) or, if it’s time-series data, interpolation is a great way to impute data.


In [ ]:
## Interpolate ##
#################
# interpolation of missing values (useful in time-series)
df.interpolate() # all dataframe
df["sepal_length"].interpolate() # specific column

#5. Subsetting & working with columns
Not all columns in the dataset are of interest, sometimes we select specific columns for analytics or building a model. Subsetting allows you to do that.
There are two key ways to select columns: by column names and by column positions:


In [ ]:
# select a column by column name
df["sepal_length"]
# select multiple columns by column name
df[["sepal_length", "sepal_width", "petal_length", "spp"]]
# select a column by column number
df.iloc[:, 2:4]
# select multiple columns by column number
df.iloc[:, [1,3,4]]

But what if you want to subset data by dropping a column?


In [ ]:
# drop a column 
df.drop("sepal_length", axis=1)


Now let’s say you want to create a new column by adding two existing columns:
sepal_len_cm= sepal_length* 10
Creating new calculated columns is often big part of feature engineering.

In [ ]:
# add new calculated column
df['new'] = df["sepal_length"]*2
# create a conditional calculated column
df['newcol'] = ["short" if i<3 else "long" for i in df["sepal_width"]] 


Sometimes re-coding may be needed to convert categorical string values to numeric values.


In [ ]:
df.replace({"Species":{"setosa":1, "versicolor":2, "virginica":3}})


If aggregation of column values is needed (mean/median etc.), python and numpy has native functions that can be applied to the dataframe.


In [ ]:
# calculate mean of each of two columns
df[["sepal_length", "sepal_width"]].mean()
# calculate sum and mean of each column
df[["sepal_length", "sepal_width"]].agg([np.sum, np.mean])

And finally, some bonus syntax useful to work with columns:


In [ ]:
# transposing a dataset
df.T
# create a list of columns
df.columns.tolist()
# sorting values in ascending order
df.sort_values(by = "sepal_width", ascending = True)
# change column name
df.rename(columns={"old_name": "new_name"})

#6. Filtering: working with rows

Filtering is an important part of exploratory data analysis, drawing insights and building KPIs.
There are many ways to filter data depending on the analytics needs, such as:
a) using the row index location:

In [ ]:
# select rows with index number 3 to 10
df.iloc[3:10,]
# select rows with index name
df.loc["index1", "index2"]
# finding rows with specific strings
df[df["species"].isin(["setosa"])]

b) conditional filtering


In [ ]:
# simple conditional filtering to filter rows with sepal_length>=5
df.query('sepal_length>=5') # or
df[df.sepal_length>= 5]
# filtering rows with multiple values e.g. 0.2, 0.3
df[df["petal_length"].isin([0.2, 0.3])]
# multi-conditional filtering
df[(df.sepal_length>1) & (df.species=="setosa") | (df.sepal_width<3)]

And finally, here’s how you’d get rid of a row if needed.


In [ ]:
# drop rows
df.drop(df.index[1]) # 1 is row index to be deleted


#7. Grouping

Like filtering, grouping is another important part of exploratory data analysis and data visualization. The key function for this task is groupby() and is mainly used for aggregating rows based on categorical features.


In [ ]:
# return a dataframe object grouped by "species" column
df.groupby("species")

After the dataframe is grouped, you could apply different functions to it, for example, getting aggregate values of numeric columns:


In [ ]:
# return mean a column groupby "species" categories
df["sepal_length"].groupby(df["species"]).mean()

Or you can apply such aggregate function to multiple features:


In [ ]:
# group each column by "species", then apply multiple operation on each feature 
df.groupby("species").agg([np.sum, np.mean, np.std])

#8. Joining/merging

If you know SQL I don’t have to explain how important joining is. Python and pandas have some functions such as merge(), join(), concat() for SQL style joining. If SQL is the primary database you probably won’t have to do joining much in Python, but nevertheless you should add the following codes to your cheatsheet.

In [ ]:
# SQL style joining
df1 = df[["sepal_length", "sepal_width"]]
df2 = df[["sepal_length", "petal_length"]]
dfx = pd.concat([df1, df2], axis = 1)